In [120]:
import jax
import jax.numpy as jnp
from jax import random

from flax import linen as nn
from typing import Callable, Any, Optional


In [121]:

class Encoder(nn.Module):

    @nn.compact
    def __call__(self, x):
        
        x = nn.Conv(512, kernel_size=(3,3),  strides=[2,2], padding='same')(x)
        x = nn.relu(x)
        x = nn.normalization.BatchNorm(True)(x)
        # x = nn.max_pool(x, window_shape=(2,2), strides=(2,2))

        
        x = nn.Conv(512,kernel_size=(3,3), strides=[2,2], padding='same')(x)
        x = nn.relu(x)
        x = nn.normalization.BatchNorm(True)(x)
        x = nn.max_pool(x, window_shape=(2,2), strides=(2,2))

        
        x = nn.Conv(256,kernel_size=(3,3), strides=[2,2],padding='same')(x)
        x = nn.relu(x)
        x = nn.normalization.BatchNorm(True)(x)
 

        x = nn.Conv(128,kernel_size=(3,3), padding='same')(x)
        x = nn.relu(x)
        x = nn.normalization.BatchNorm(True)(x)
        
        x = nn.Conv(64,kernel_size=(3,3), strides=[2,2], padding='same')(x)
        x = nn.relu(x)
        x = nn.normalization.BatchNorm(True)(x)
        x = nn.max_pool(x, window_shape=(2,2), strides=(2,2))

        
        x = nn.Conv(32, kernel_size=(3, 3),  padding='same')(x)
        x = nn.relu(x)
        x = nn.normalization.BatchNorm(True)(x)
        
        
        x = nn.Conv(16, kernel_size=(2,2), strides=[1,1], padding='same')(x)
        x = nn.relu(x)
        x = nn.normalization.BatchNorm(True)(x)
        
        x = nn.Conv(1,kernel_size=(2,2), strides=[1,1],  padding='same')(x)
        x = nn.relu(x)
        x = nn.normalization.BatchNorm(True)(x)

        
        x = x.reshape(x.shape[0], -1)
        
        
        mean_x = nn.Dense(20, name='fc3_mean')(x)
        logvar_x = nn.Dense(20, name='fc3_logvar')(x)
        
        
        return mean_x, logvar_x

In [122]:
enc = Encoder()

In [123]:
rng = jax.random.PRNGKey(303)

In [124]:
import numpy as np

In [213]:
x = np.random.randn(16, 48, 1876,1)

In [214]:
init_vari = enc.init(rng, x)

In [215]:
init_vari = enc.init(rng, x)
nn.tabulate(enc, rngs={'params': rng})(x)

                           Encoder Summary                            
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path       ┃ outputs                ┃ params                       ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Inputs     │ float64[16,48,1876,1]  │                              │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_0     │ float32[16,24,938,512] │ bias: float32[512]           │
│            │                        │ kernel: float32[3,3,1,512]   │
│            │                        │                              │
│            │                        │ 5,120 (20.5 KB)              │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_1     │ float32[16,12,469,512] │ bias: float32[512]           │
│            │                        │ kernel: float32[3,3,512,512] │
│            │                        │                              │
│            │                        │ 2,359,808 (9.4 MB)           │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_2     │ float32[16,6,235,256]  │ bias: float32[256]           │
│            │                        │ kernel: float32[3,3,512,256] │
│            │                        │                              │
│            │                        │ 1,179,904 (4.7 MB)           │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_3     │ float32[16,6,235,128]  │ bias: float32[128]           │
│            │                        │ kernel: float32[3,3,256,128] │
│            │                        │                              │
│            │                        │ 295,040 (1.2 MB)             │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_4     │ float32[16,3,118,64]   │ bias: float32[64]            │
│            │                        │ kernel: float32[3,3,128,64]  │
│            │                        │                              │
│            │                        │ 73,792 (295.2 KB)            │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_5     │ float32[16,3,118,32]   │ bias: float32[32]            │
│            │                        │ kernel: float32[3,3,64,32]   │
│            │                        │                              │
│            │                        │ 18,464 (73.9 KB)             │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_6     │ float32[16,2,59,16]    │ bias: float32[16]            │
│            │                        │ kernel: float32[2,2,32,16]   │
│            │                        │                              │
│            │                        │ 2,064 (8.3 KB)               │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_7     │ float32[16,1,30,8]     │ bias: float32[8]             │
│            │                        │ kernel: float32[2,2,16,8]    │
│            │                        │                              │
│            │                        │ 520 (2.1 KB)                 │
├────────────┼────────────────────────┼──────────────────────────────┤
│ fc3_logvar │ float32[16,20]         │ bias: float32[20]            │
│            │                        │ kernel: float32[240,20]      │
│            │                        │                              │
│            │                        │ 4,820 (19.3 KB)              │
├────────────┼────────────────────────┼──────────────────────────────┤
│ fc3_mean   │ float32[16,20]         │ bias: float32[20]            │
│            │                        │ kernel: float32[240,20]      │
│            │                        │                              │
│            │                        │ 4,820 (19.3 KB)              │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Encoder    │ - float32

'\n\n'

In [216]:

class Encoder(nn.Module):

    @nn.compact
    def __call__(self, x):
        
        x = nn.Conv(512, kernel_size=(3,3),  strides=[2,2], padding='same')(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)
        # x = nn.max_pool(x, window_shape=(2,2), strides=(2,2))

        
        x = nn.Conv(512,kernel_size=(3,3), strides=[2,2], padding='same')(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)
        # x = nn.max_pool(x, window_shape=(2,2), strides=(2,2))

        
        x = nn.Conv(256,kernel_size=(3,3), strides=[2,2],padding='same')(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)
 

        x = nn.Conv(128,kernel_size=(3,3), padding='same')(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)
        
        x = nn.Conv(64,kernel_size=(3,3), strides=[2,2], padding='same')(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)
        # x = nn.max_pool(x, window_shape=(2,2), strides=(2,2))

        
        x = nn.Conv(32, kernel_size=(3, 3),  padding='same')(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)
        
        
        x = nn.Conv(16, kernel_size=(2,2), strides=[2,2], padding='same')(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)
        
        x = nn.Conv(8,kernel_size=(2,2), strides=[2,2],  padding='same')(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)

        
        x = x.reshape(x.shape[0], -1)
        
        
        mean_x = nn.Dense(20, name='fc3_mean')(x)
        logvar_x = nn.Dense(20, name='fc3_logvar')(x)
        
        
        return mean_x, logvar_x

In [217]:
enc = Encoder()

In [218]:
init_vari = enc.init(rng, x)
nn.tabulate(enc, rngs={'params': rng})(x)

                           Encoder Summary                            
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path       ┃ outputs                ┃ params                       ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Inputs     │ float64[16,48,1876,1]  │                              │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_0     │ float32[16,24,938,512] │ bias: float32[512]           │
│            │                        │ kernel: float32[3,3,1,512]   │
│            │                        │                              │
│            │                        │ 5,120 (20.5 KB)              │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_1     │ float32[16,12,469,512] │ bias: float32[512]           │
│            │                        │ kernel: float32[3,3,512,512] │
│            │                        │                              │
│            │                        │ 2,359,808 (9.4 MB)           │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_2     │ float32[16,6,235,256]  │ bias: float32[256]           │
│            │                        │ kernel: float32[3,3,512,256] │
│            │                        │                              │
│            │                        │ 1,179,904 (4.7 MB)           │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_3     │ float32[16,6,235,128]  │ bias: float32[128]           │
│            │                        │ kernel: float32[3,3,256,128] │
│            │                        │                              │
│            │                        │ 295,040 (1.2 MB)             │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_4     │ float32[16,3,118,64]   │ bias: float32[64]            │
│            │                        │ kernel: float32[3,3,128,64]  │
│            │                        │                              │
│            │                        │ 73,792 (295.2 KB)            │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_5     │ float32[16,3,118,32]   │ bias: float32[32]            │
│            │                        │ kernel: float32[3,3,64,32]   │
│            │                        │                              │
│            │                        │ 18,464 (73.9 KB)             │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_6     │ float32[16,2,59,16]    │ bias: float32[16]            │
│            │                        │ kernel: float32[2,2,32,16]   │
│            │                        │                              │
│            │                        │ 2,064 (8.3 KB)               │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Conv_7     │ float32[16,1,30,8]     │ bias: float32[8]             │
│            │                        │ kernel: float32[2,2,16,8]    │
│            │                        │                              │
│            │                        │ 520 (2.1 KB)                 │
├────────────┼────────────────────────┼──────────────────────────────┤
│ fc3_logvar │ float32[16,20]         │ bias: float32[20]            │
│            │                        │ kernel: float32[240,20]      │
│            │                        │                              │
│            │                        │ 4,820 (19.3 KB)              │
├────────────┼────────────────────────┼──────────────────────────────┤
│ fc3_mean   │ float32[16,20]         │ bias: float32[20]            │
│            │                        │ kernel: float32[240,20]      │
│            │                        │                              │
│            │                        │ 4,820 (19.3 KB)              │
├────────────┼────────────────────────┼──────────────────────────────┤
│ Encoder    │ - float32

'\n\n'

In [159]:
x = np.random.randn(16,20)

In [202]:
class Decoder(nn.Module):
    
    recon_shape: int = 1876
    
    @nn.compact
    def __call__(self, x):
        
        x = nn.Dense(2 * 59)(x)        
        x = x.reshape(x.shape[0], 2, 59, 1)
        
        x = nn.ConvTranspose(64, kernel_size=(1,3), padding='valid')(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)
        
        x = nn.ConvTranspose(128, kernel_size=(3,3), strides=[2,2])(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)        
        
        x = nn.ConvTranspose(128, kernel_size=(3,3), strides=[2,2])(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)
        
        x = nn.ConvTranspose(256, kernel_size=(3,3), strides=[2,2])(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)
        
        x = nn.ConvTranspose(256, kernel_size=(3,3), strides=[2,2])(x)
        x = nn.relu(x)
        # x = nn.normalization.BatchNorm(True)(x)
        
        x = nn.ConvTranspose(512, kernel_size=(3,3))(x)
        x = nn.relu(x)
        
        x = nn.ConvTranspose(1, kernel_size=(3,3), strides=[2,2])(x)
        x = nn.relu(x)

        return x

In [203]:
dec = Decoder()

In [204]:
x.shape

(16, 20)

In [205]:
initial_vari = dec.init(rng, x = x)

In [206]:
nn.tabulate(dec, rngs={'params': rng})(x)

                              Decoder Summary                              
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path            ┃ outputs                ┃ params                       ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Inputs          │ float64[16,20]         │                              │
├─────────────────┼────────────────────────┼──────────────────────────────┤
│ ConvTranspose_0 │ float32[16,2,57,64]    │ bias: float32[64]            │
│                 │                        │ kernel: float32[1,3,1,64]    │
│                 │                        │                              │
│                 │                        │ 256 (1.0 KB)                 │
├─────────────────┼────────────────────────┼──────────────────────────────┤
│ ConvTranspose_1 │ float32[16,4,114,128]  │ bias: float32[128]           │
│                 │                        │ kernel: float32[3,3,64,128]  │
│                 │                        │                              │
│                 │                        │ 73,856 (295.4 KB)            │
├─────────────────┼────────────────────────┼──────────────────────────────┤
│ ConvTranspose_2 │ float32[16,8,228,128]  │ bias: float32[128]           │
│                 │                        │ kernel: float32[3,3,128,128] │
│                 │                        │                              │
│                 │                        │ 147,584 (590.3 KB)           │
├─────────────────┼────────────────────────┼──────────────────────────────┤
│ ConvTranspose_3 │ float32[16,16,456,256] │ bias: float32[256]           │
│                 │                        │ kernel: float32[3,3,128,256] │
│                 │                        │                              │
│                 │                        │ 295,168 (1.2 MB)             │
├─────────────────┼────────────────────────┼──────────────────────────────┤
│ ConvTranspose_4 │ float32[16,32,912,256] │ bias: float32[256]           │
│                 │                        │ kernel: float32[3,3,256,256] │
│                 │                        │                              │
│                 │                        │ 590,080 (2.4 MB)             │
├─────────────────┼────────────────────────┼──────────────────────────────┤
│ ConvTranspose_5 │ float32[16,32,912,512] │ bias: float32[512]           │
│                 │                        │ kernel: float32[3,3,256,512] │
│                 │                        │                              │
│                 │                        │ 1,180,160 (4.7 MB)           │
├─────────────────┼────────────────────────┼──────────────────────────────┤
│ ConvTranspose_6 │ float32[16,64,1824,1]  │ bias: float32[1]             │
│                 │                        │ kernel: float32[3,3,512,1]   │
│                 │                        │                              │
│                 │                        │ 4,609 (18.4 KB)              │
├─────────────────┼────────────────────────┼──────────────────────────────┤
│ Dense_0         │ float32[16,118]        │ bias: float32[118]           │
│                 │                        │ kernel: float32[20,118]      │
│                 │                        │                              │
│                 │                        │ 2,478 (9.9 KB)               │
├─────────────────┼────────────────────────┼──────────────────────────────┤
│ Decoder         │ float32[16,64,1824,1]  │                              │
├─────────────────┼────────────────────────┼──────────────────────────────┤
│                 │                  Total │ 2,294,191 (9.2 MB)           │
└─────────────────┴────────────────────────┴──────────────────────────────┘
                                                                           
                   Total Parameters: 2,294,191 (9.2 MB)                    

'\n\n'